In [ ]:
from __future__ import print_function, division
from builtins import range, input

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from keras.layers import Dropout, BatchNormalization, Activation
from keras.regularizers import l2
from keras.optimizers import SGD, Adam

# Veriyi yükleyelim.
df = pd.read_csv('edited_rating.csv')

# N kullanıcıların sayısı
N = df.userId.max() + 1
# Filmlerin sayısı
M = df.movie_idx.max() + 1

# Veriyi test ve eğitim verisi olarak ayıralım.
df = shuffle(df)
cutoff = int(0.8*len(df))
df_train = df.iloc[:cutoff]
df_test = df.iloc[cutoff:]

K = 10 # Gizli boyutluluk(Latent Dimensionality)
mu = df_train.rating.mean()
epochs = 15
#reg = 0.0001 # regularizasyon uygulayarak maliyet fonksiyonuna ekstra bir terim ekleyerek maliyeti artırmak için kullanıyoruz.
#Regularization sayesinde overfittingi yani ezberleme problemini engelleyebiliriz.


# keras modelimiz.
u = Input(shape=(1,))
m = Input(shape=(1,))
#Embedding fonksiyonu, verinin yüksek boyutlu uzayını daha düşük boyutlu bir uzaya dönüştürerek boyut indirgeme sağlar.
u_embedding = Embedding(N, K)(u) # (N, 1, K)
m_embedding = Embedding(N, K)(m) # (N, 1, K)
#Boyutlarını indirgedikten sonra inputlarımızı düzleştirelim.Böylece modelimizi eğitebiliriz.
u_embedding = Flatten()(u_embedding) # (N, K)
m_embedding = Flatten()(m_embedding) # (N, K)
#Concantinate, birleştirme demek. Aslında her bir yoldan gelen, özellik haritalarını birleştiriyor. Buna concantinate deniyor.
x = Concatenate()([u_embedding, m_embedding]) # (N, 2K)
#Artık x input olarak hazır.

# Modelimizi tasarlayalım.400'lük bir Dense katmanı oluşturduk.Girdi olarak x'i aldı.X'i girdileri düzenleyerek elde etmiştik.
x = Dense(400)(x)
#Burada BatchNormalization'da yapabiliriz.BatchNormalization sinir ağlarının eğitimini hızlandırır,
#daha iyi bir genelleme sağlar ve modelin daha kararlı ve güvenilir olmasını sağlar
# x = BatchNormalization()(x)
x = Activation('relu')(x)

#Dropout'ta tıpkı Regularization gibi overfittingi engelleme yöntemlerinden birisi.
#Modelimize 100'lük bir dense katmanı daha ekleyerek modeli daha karmaşık hale getirebiliriz.
#Bu sayede model daha iyi öğrenebilir.Bunu deneyerek test etmeliyiz.
# x = Dense(100)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
x = Dense(1)(x)

#Modelimize girdileri ve çıktıyı veriyoruz.
model = Model(inputs=[u, m], outputs=x)
#Modeli derliyoruz.Optimizer olarak adam'ı kullandık.
#Adam yerine sgd optimizeri'da deneyerek hangisi daha iyi sonuç veriyor görebiliriz.
model.compile(
  loss='mse',
  # optimizer='adam',
  # optimizer=Adam(lr=0.01),
  optimizer='adam',
  metrics=['mse'],
)

#Modeli eğitmeye başlıyoruz.128'lik batchlar halinde veriyi alıp işliyor.
#Modeli 15 epochta eğiteceğiz.
r = model.fit(
  x=[df_train.userId.values, df_train.movie_idx.values],
  y=df_train.rating.values - mu,
  epochs=epochs,
  batch_size=128,
  validation_data=(
    [df_test.userId.values, df_test.movie_idx.values],
    df_test.rating.values - mu
  )
)


# modelin test ve eğitim hata sonuçlarını tabloda görelim.
plt.plot(r.history['loss'], label="train loss")
plt.plot(r.history['val_loss'], label="test loss")
plt.legend()
plt.show()